In [1]:
import json
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity import UnityEnvironments
from unity_sds_client.unity_session import UnitySession
from unity_sds_client.unity_services import UnityServices as services
from unity_sds_client.resources.collection import Collection

In [2]:
# We will set the environment to 'DEV' here but this should be set to test or prod eventually.
s = Unity(UnityEnvironments.TEST)
# set the venue for interacting with venue specific services
# if your venue id is a single string, use the following

Please enter your Unity username:  gangl
Please enter your Unity password:  ········


In [3]:
dataManager = s.client(services.DATA_SERVICE)
collection_id = "urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1"
cd = dataManager.get_collection_data(Collection(collection_id))
for dataset in cd:
    print(dataset.data_create_time)
    print(f'dataset name: {dataset.data_begin_time}')
    print(f'dataset name: {dataset.id}' )
    print(f'dataset soil props: ' + str(dataset.properties['soil']) )
    for f in dataset.datafiles:
        print(f)
        #print("	" + f.location + ", roles: " + str(f.roles) + ", type: " + f.type + ", description: " + f.description + ", title: " + f.title)

2023-08-07T18:27:55Z
dataset name: 2023-08-07T18:27:55Z
dataset name: urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1:SISTER_EMIT_L2B_FRCOV_20230807T182755_001
dataset soil props: {'0.0': 1862734.0, '0.1': 1508139.0, '0.2': 1293726.0, '0.3': 1063639.0, '0.4': 839482.0, '0.5': 629223.0, '0.6': 434886.0, '0.7': 261035.0, '0.8': 135612.0, '0.9': 63628.0, '1.0': 22433.0}
unity_sds_client.resources.DataFile(location=s3://sps-test-ds-storage/urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1/urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1:SISTER_EMIT_L2B_FRCOV_20230807T182755_001/SISTER_EMIT_L2B_FRCOV_20230807T182755_001.cmr.xml)
unity_sds_client.resources.DataFile(location=s3://sps-test-ds-storage/urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1/urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1:SISTER_EMIT_L2B_FRCOV_20230807T182755_001/SISTER_EMIT_L2B_FRCOV_20230807T182755_001.json)
unity_sds_client.resources.DataFile(location=s3://sps-test-ds-storage/urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1/urn:nasa:unity:unity:

In [4]:
# Check the metadata for soil cover
key = "soil"
threshold = '0.5'
pixel_count = 100

for dataset in cd:
    if dataset.properties[key][threshold] > pixel_count:
        print(f'Execute l2 algorithm for {dataset.id}')

Execute l2 algorithm for urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1:SISTER_EMIT_L2B_FRCOV_20230807T182755_001


In [5]:
# Execute the L2 workflow

In [6]:
import requests
import getpass

username = getpass.getpass('Enter your username')
password = getpass.getpass('Enter your password')

data = {
  "conf": {
    "cwl_workflow":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.cwl",
    "args_as_json":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.test.yml"
  },
  "note": "cli run"
}
# Will be replaced by the WPS execute scripts
# Auth: Only needed until we get the cognito user stuff worked out,
response = requests.post("http://k8s-airflow-airflowi-5b65901547-1494561622.us-west-2.elb.amazonaws.com:5000/api/v1/dags/cwl-dag/dagRuns",auth=(username, password), headers={"accept":"application/json", "Content-Type":"application/json"}, json=data)
response.json()

Enter your username ········
Enter your password ········


{'conf': {'args_as_json': 'https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.test.yml',
  'cwl_workflow': 'https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.cwl'},
 'dag_id': 'cwl-dag',
 'dag_run_id': 'manual__2024-03-26T23:04:37.355521+00:00',
 'data_interval_end': '2024-03-26T23:04:37.355521+00:00',
 'data_interval_start': '2024-03-26T23:04:37.355521+00:00',
 'end_date': None,
 'execution_date': '2024-03-26T23:04:37.355521+00:00',
 'external_trigger': True,
 'last_scheduling_decision': None,
 'logical_date': '2024-03-26T23:04:37.355521+00:00',
 'note': 'cli run',
 'run_type': 'manual',
 'start_date': None,
 'state': 'queued'}